In [1]:
from IPython.display import display, Image

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [3]:
from AutoScreen.AndroidScreen import ADBPropertiesHelper, AndroidController, UINode

import cv2
import xml.etree.ElementTree as ET
from script.utils import add_text_with_transparent_bg, draw_bbox_multi, AndroidUIHelper
import os
import shutil
import time

DEBUG:matplotlib:matplotlib data path: D:\FullStack\python\conda\envs\py312asp\Lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\Admin\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32


In [4]:
temp_data_base_dir = os.path.join("data", "temp")
# shutil.rmtree(temp_data_base_dir)
# os.makedirs(temp_data_base_dir)

In [5]:
for round in range(10):
    logging.info(f"\n\n---\n\ngetting wechat moment round {round}")
    # try to get screenshot and ui xml
    for _ in range(3):
        try:
            android_controller = AndroidController()
            screenshot_raw_path = android_controller.get_screenshot("wechat_moment", temp_data_base_dir)
            xml_raw_path = android_controller.get_xml("wechat_moment", temp_data_base_dir)
            logging.info(f"Successfully saved screenshot and ui xml for {android_controller.device}")
            break
        except RuntimeError as e:
            android_controller.reopen_app_by_going_to_homescreen()
    # display(Image(screenshot_raw_path))
    
    # parse xml tree
    tree = ET.parse(xml_raw_path)
    ui_node_list = []
    for ele in tree.iter():
        if ele.attrib.get("clickable", "false") == "true":
        # if ele.attrib.get("clickable", "false") == "true" or ele.attrib.get("focusable", "false") == "true":   
            left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=ele.attrib.get("bounds"))
            name = ele.attrib.get("resource-id", "no-resource-id").split('/')[-1]
            if name is None:
                name = ele.attrib.get("content-desc", "no-content-desc")
            ui_node_list.append(UINode(name, ((left, high), (right, low)), ele.attrib))
    
    # draw parsed nodes on the screenshot
    draw_bbox_multi(screenshot_raw_path, temp_data_base_dir + "/with_box.png", ui_node_list)
    # display(Image(temp_data_base_dir + "/with_box.png"))
    
    # get moment list
    moment_node_list = tree.findall(".//*[@resource-id='com.tencent.mm:id/n9a']") + tree.findall(".//*[@resource-id='com.tencent.mm:id/n9e']")
    logging.info(f"found {len(moment_node_list)} n9a / n9e node")
    if len(moment_node_list) == 0:
        logging.error("no moment found, try reopen_app_by_going_to_homescreen...")
        android_controller.reopen_app_by_going_to_homescreen()
        continue
    
    # get the first good moment
    this_moment_low = None
    this_moment_node = None
    for i, moment_node_i in enumerate(moment_node_list):
        left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=moment_node_i.attrib.get("bounds"))
        # logging.info("this_moment_node is with a high that is too small, skipping it.")
        logging.info(f"moment node {i}, high={high}, low={low}")
        if high > 192 and low < 1918:
            logging.info(f"choosing moment node {i}, with all the box visible")
            this_moment_low = low
            this_moment_node = moment_node_i
            break
    if this_moment_low is None or this_moment_node is None:
        logging.warning("no suitable moment found, skip this round and swipe up a little bit")
        android_controller.swipe_up(200)
        continue
        
    
    # save this moment raw data
    this_moment_tree = ET.ElementTree(this_moment_node)
    this_moment_tree.write(temp_data_base_dir + "/this_moment_tree.xml")

    this_moment_is_ad = False
    # parse node one by one, by its type
    for i, node in enumerate(this_moment_tree.iter()):
        resource_id = node.attrib.get("resource-id", "no-resource-id")
        content_desc = node.attrib.get("content-desc", "no-content-desc")
        text = node.attrib.get("text", "no-text")
        left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=node.attrib.get("bounds"))
        logging.debug(f"itering within child nodes, {i}, {content_desc}, {text}, {left}, {high}, {right}, {low}")
        if resource_id == "com.tencent.mm:id/n9a":
            logging.debug(f"skipping this node {i}, it's a n9a, 朋友圈主node")
        elif resource_id == "com.tencent.mm:id/n9e":
            logging.debug(f"skipping this node {i}, it's a n9e, 朋友圈主node")
        elif resource_id == "com.tencent.mm:id/od":
            this_avatar_image = cv2.imread(screenshot_raw_path)[high:low, left:right]
            cv2.imwrite(temp_data_base_dir + "/this_avatar_image.png", this_avatar_image)
            # display(Image(temp_data_base_dir + "/this_avatar_image.png"))
            logging.info(f"found od, left={left}, right={right}, high={high}, low={low}")
        elif "图片第" in content_desc:
            # TODO: 保存图片
            # print(left, right, high, low)
            logging.info(f"found picture, left={left}, right={right}, high={high}, low={low}")
            android_controller.tap_in_box(left=left, high=high, right=right, low=low)  # open the picture
            try:
                time.sleep(10)  # make sure the picture is loaded
                android_controller.long_press(android_controller.width//2, android_controller.height//2)
                picture_xml_path = android_controller.get_xml("wechat_moment_picture", temp_data_base_dir)  # save the ui xml
                picture_tree = ET.parse(picture_xml_path)
                save_node = picture_tree.find(".//*[@text='保存图片']")
                left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=save_node.attrib.get("bounds"))
                android_controller.tap_in_box(left=left, high=high, right=right, low=low)  # click save
                time.sleep(3)  # make sure the picture is saved
                this_picture_path = android_controller.copy_and_remove_the_latest_file("/sdcard/Pictures/WeiXin", temp_data_base_dir)
                logging.info(f"saved picture, this_picture_path={this_picture_path}")
                print(this_picture_path)
            except Exception as e:
                logging.warning(f"saved picture error: {e}")
                pass
            finally:
                android_controller.press_back_key()
        elif resource_id == "com.tencent.mm:id/n96":
            n96 = text
            if len(n96) > 0:
                # TODO: 被折叠的朋友圈
                print(n96)
                logging.info(f"found n96, 被折叠的朋友圈 is {n96}")
            else:
                logging.debug(f"skipping this node {i}, it's a n96 and length is 0, 被折叠的朋友圈")
        elif resource_id == "com.tencent.mm:id/kbq":
            kbq = text
            # TODO: 保存用户
            print(kbq)
            logging.info(f"found kbq, user is {kbq}")
        elif resource_id == "com.tencent.mm:id/h9t":
            logging.debug(f"skipping this node {i}, it's a h9t, odj的父节点")
        elif resource_id == "com.tencent.mm:id/odj":
            odj = text
            # TODO: 保存分享链接标题
            print(odj)
            logging.info(f"found odj, 分享链接标题 is {odj}")
        elif resource_id == "com.tencent.mm:id/cuf" or resource_id == "com.tencent.mm:id/cut":
            cuf_cut = text
            if len(cuf_cut) > 0:
                # TODO: 保存文字
                print(cuf_cut)
                logging.info(f"found cuf_cut, 朋友圈内容文字 is {cuf_cut}")
            else:
                logging.debug(f"skipping this node {i}, it's a cuf / cut and length is 0")
        elif resource_id == "com.tencent.mm:id/r2":
            logging.debug(f"skipping this node {i}, it's a r2, 评论区图标")
        elif resource_id == "com.tencent.mm:id/hbr":
            logging.debug(f"skipping this node {i}, it's a hbr, 点赞区图标")
        elif resource_id == "com.tencent.mm:id/n9v":
            n9v = text
            # TODO: 保存点赞的人
            print(n9v)
            logging.info(f"found cut, 点赞的人 is {n9v}")
        elif resource_id == "com.tencent.mm:id/c6p":
            c6p = text
            # TODO: 保存此条评论
            print(c6p)
            logging.info(f"found c6p, 此条评论 is {c6p}")
        elif text is not None and len(text) > 0:
            if node.attrib.get("clickable", "false") == "true":
                # TODO: 保存地址?
                logging.info(f"found extra text, maybe location, text is {text}")
                if text == "广告":
                    this_moment_is_ad = True
                    logging.warning(f"found out this is ad")
            else:
                # TODO: 保存日期?
                logging.info(f"found extra text, maybe a datetime or location, text is {text}")
        else:
            logging.debug(f"ignoring this node, {i}, {node.attrib}")
    
    # next swipe up
    next_swipe_up_distance = this_moment_low - 240
    logging.info(f"finished this moment, moving to the next one, next_swipe_up_distance={next_swipe_up_distance}")
    
    android_controller.swipe_up(next_swipe_up_distance)

logging.warning("\n\n---\n\nprogam exit")

INFO:root:

---

getting wechat moment round 0
INFO:root:run adb command: adb devices
DEBUG:root:adb command return with this stdout: List of devices attached
f33025a4	device
INFO:root:get android devices: ['f33025a4']
INFO:root:no device provided in AndroidController.__init__, using f33025a4
INFO:root:run adb command: adb -s f33025a4 shell wm size
DEBUG:root:adb command return with this stdout: Physical size: 1080x1920
INFO:root:run adb command: adb -s f33025a4 shell wm size
DEBUG:root:adb command return with this stdout: Physical size: 1080x1920
INFO:root:run adb command: adb -s f33025a4 shell screencap -p /sdcard/Documents/ASP/wechat_moment.png
INFO:root:run adb command: adb -s f33025a4 pull /sdcard/Documents/ASP/wechat_moment.png data\temp\wechat_moment.png
DEBUG:root:adb command return with this stdout: /sdcard/Documents/ASP/wechat_moment.png: 1 file pulled, 0 skipped. 17.7 MB/s (769207 bytes in 0.041s)
INFO:root:run adb command: adb -s f33025a4 shell rm /sdcard/Documents/ASP/wech

366 A-温哥华岛华人信息1
【房源】出租全新公寓，Mayfair mall附近一套一室公寓出租，边户，光线特别好，有一停车位，公寓附近购物方便，公交车可直达各个学校，不带家具，即日起出租，有意者联系微信 lvben1985
​​​​​​​#加拿大维多利亚房源信息#温哥华岛传媒


INFO:root:swipe called, start_x=540, start_y=960, end_x=545, end_y=950, duration=1499
INFO:root:run adb command: adb -s f33025a4 shell input swipe 540 960 545 950 1499
INFO:root:run adb command: adb -s f33025a4 shell uiautomator dump --compressed /sdcard/Documents/ASP/wechat_moment_picture.xml
DEBUG:root:adb command return with this stdout: UI hierchary dumped to: /sdcard/Documents/ASP/wechat_moment_picture.xml
INFO:root:run adb command: adb -s f33025a4 pull /sdcard/Documents/ASP/wechat_moment_picture.xml data\temp\wechat_moment_picture.xml
DEBUG:root:adb command return with this stdout: /sdcard/Documents/ASP/wechat_moment_picture.xml: 1 file pulled, 0 skipped. 3.3 MB/s (12363 bytes in 0.004s)
INFO:root:run adb command: adb -s f33025a4 shell rm /sdcard/Documents/ASP/wechat_moment_picture.xml
INFO:root:tap called, x=176, y=1687
INFO:root:run adb command: adb -s f33025a4 shell input tap 176 1687
INFO:root:run adb command: adb -s f33025a4 shell ls /sdcard/Pictures/WeiXin -tp | grep -v /$ 

data\temp\mmexport1706612454906.jpg


DEBUG:root:itering within child nodes, 6, 图片第2张共2张, , 464, 1114, 731, 1381
INFO:root:found picture, left=464, right=731, high=1114, low=1381
INFO:root:tap called, x=626, y=1298
INFO:root:run adb command: adb -s f33025a4 shell input tap 626 1298
INFO:root:swipe called, start_x=540, start_y=960, end_x=555, end_y=940, duration=1916
INFO:root:run adb command: adb -s f33025a4 shell input swipe 540 960 555 940 1916
INFO:root:run adb command: adb -s f33025a4 shell uiautomator dump --compressed /sdcard/Documents/ASP/wechat_moment_picture.xml
DEBUG:root:adb command return with this stdout: UI hierchary dumped to: /sdcard/Documents/ASP/wechat_moment_picture.xml
INFO:root:run adb command: adb -s f33025a4 pull /sdcard/Documents/ASP/wechat_moment_picture.xml data\temp\wechat_moment_picture.xml
DEBUG:root:adb command return with this stdout: /sdcard/Documents/ASP/wechat_moment_picture.xml: 1 file pulled, 0 skipped. 2.7 MB/s (11661 bytes in 0.004s)
INFO:root:run adb command: adb -s f33025a4 shell rm 

...

DEBUG:root:itering within child nodes, 6, , 昨天, 189, 1054, 265, 1098
INFO:root:found extra text, maybe a datetime or location, text is 昨天
DEBUG:root:itering within child nodes, 7, , , 956, 1049, 1042, 1103
DEBUG:root:skipping this node 7, it's a r2, 评论区图标
INFO:root:finished this moment, moving to the next one, next_swipe_up_distance=886
INFO:root:swipe_up called, diff_height=886
INFO:root:swipe called, start_x=448, start_y=1720, end_x=231, end_y=834, duration=5088
INFO:root:run adb command: adb -s f33025a4 shell input swipe 448 1720 231 834 5088
INFO:root:

---

getting wechat moment round 9
INFO:root:run adb command: adb devices
DEBUG:root:adb command return with this stdout: List of devices attached
f33025a4	device
INFO:root:get android devices: ['f33025a4']
INFO:root:no device provided in AndroidController.__init__, using f33025a4
INFO:root:run adb command: adb -s f33025a4 shell wm size
DEBUG:root:adb command return with this stdout: Physical size: 1080x1920
INFO:root:run adb comman

365 阿波罗🔥维多利亚便民服务站
【♻️出闲置】9成新小冰箱 需要自提， 可小刀，联系微信：Y427120


INFO:root:swipe called, start_x=540, start_y=960, end_x=539, end_y=976, duration=1459
INFO:root:run adb command: adb -s f33025a4 shell input swipe 540 960 539 976 1459
INFO:root:run adb command: adb -s f33025a4 shell uiautomator dump --compressed /sdcard/Documents/ASP/wechat_moment_picture.xml
DEBUG:root:adb command return with this stdout: UI hierchary dumped to: /sdcard/Documents/ASP/wechat_moment_picture.xml
INFO:root:run adb command: adb -s f33025a4 pull /sdcard/Documents/ASP/wechat_moment_picture.xml data\temp\wechat_moment_picture.xml
DEBUG:root:adb command return with this stdout: /sdcard/Documents/ASP/wechat_moment_picture.xml: 1 file pulled, 0 skipped. 1.3 MB/s (11661 bytes in 0.009s)
INFO:root:run adb command: adb -s f33025a4 shell rm /sdcard/Documents/ASP/wechat_moment_picture.xml
INFO:root:tap called, x=59, y=1689
INFO:root:run adb command: adb -s f33025a4 shell input tap 59 1689
INFO:root:run adb command: adb -s f33025a4 shell ls /sdcard/Pictures/WeiXin -tp | grep -v /$ | 

data\temp\mmexport1706612612484.jpg


DEBUG:root:itering within child nodes, 6, , 昨天, 189, 1048, 265, 1092
INFO:root:found extra text, maybe a datetime or location, text is 昨天
DEBUG:root:itering within child nodes, 7, , , 956, 1043, 1042, 1097
DEBUG:root:skipping this node 7, it's a r2, 评论区图标
INFO:root:finished this moment, moving to the next one, next_swipe_up_distance=880
INFO:root:swipe_up called, diff_height=880
INFO:root:swipe called, start_x=251, start_y=1866, end_x=232, end_y=986, duration=8075
INFO:root:run adb command: adb -s f33025a4 shell input swipe 251 1866 232 986 8075
